In [1]:
import caffe
from caffe.proto import caffe_pb2
from caffe import layers as L, to_proto
from caffe import params as P
import numpy as np
import random, sys, time, os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
GPU_id = 0
caffe.set_mode_gpu()
caffe.set_device(GPU_id)

In [ ]:
#Data Parameters....
CROP_SIZE = 96 # REPLACE
BATCH_SIZE = 128 #REPLACE
IMAGE_ROOT = "data/images/" #REPLACE
BIAS_CONSTANT = 0
DISCRIMINATOR_NET_PATH = "discriminator-net.prototxt"
GENERATOR_NET_PATH = "generator-net.prototxt"
SOLVER_PATH = "solver.prototxt"

#Learning Parameters
weight_param = dict(lr_mult=1, decay_mult=1)
bias_param   = dict(lr_mult=2, decay_mult=0)
learned_param = [weight_param, bias_param]
conv_filler = dict(type='xavier')
const_filler = dict(type='constant', value=BIAS_CONSTANT)
fc_filler = dict(type='xavier')

#Net Parameters (following TensorFlow implementation)
gf_dim = 64 # Dimension of gen filters in first conv layer. [64]
df_dim = 64 # Dimension of discrim filters in first conv layer. [64]
dfc_dim = 1024 # Dimension of discrim units for fully connected layer. [1024]
gfc_dim = 1024 # Dimension of gen units for fully connected layer. [1024]

In [ ]:
def conv_relu_with_batchnorm(bottom, ks, nout, stride=1, pad=1, group=1,weight_filler=conv_filler, bias_filler=const_filler):
    conv = L.Convolution(bottom,
                         kernel_size=ks,
                         num_output=nout,
                         stride=stride,
                         pad=pad,
                         group=group,
                         param=learned_param,
                         weight_filler=weight_filler,
                         bias_filler=bias_filler)
    bn = L.BatchNorm(conv,param=[dict(lr_mult=0)] * 3) # what is this @Vash
    relu = L.ReLU(bn, in_place=True)
    return conv, relu

In [ ]:
def conv_relu(bottom, ks, nout, stride=1, pad=1, group=1,weight_filler=conv_filler, bias_filler=const_filler):
    conv = L.Convolution(bottom,
                         kernel_size=ks,
                         num_output=nout,
                         stride=stride,
                         pad=pad,
                         group=group,
                         param=learned_param,
                         weight_filler=weight_filler,
                         bias_filler=bias_filler)
    relu = L.ReLU(conv, in_place=True)
    return conv, relu

In [ ]:
def fc_linear(bottom, nout,weight_filler=fc_filler, bias_filler=const_filler):
    fc = L.InnerProduct(bottom,
                        num_output=nout,
                        param=learned_param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    return fc

In [ ]:
def fc_relu(bottom, nout,weight_filler=fc_filler, bias_filler=const_filler):
    fc = L.InnerProduct(bottom,
                        num_output=nout,
                        param=learned_param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    relu = L.ReLU(fc, in_place=True)
    return fc, relu

In [ ]:
def fc_sigmoid(bottom, nout,weight_filler=fc_filler, bias_filler=const_filler):
    fc = L.InnerProduct(bottom,
                        num_output=nout,
                        param=learned_param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    relu = L.Sigmoid(fc, in_place=True)
    return fc, relu

In [ ]:
def reshape(bottom, dim1, dim2, dim3, dim4=-1):
    param = {shape: [dim1, dim2, dim3, dim4]} # check syntax
    r = L.Reshape(bottom,
                  reshape_param=param)
    return r

In [ ]:
def deconv2d_relu_batchnorm(bottom, ks, nout, stride=2, pad=1, group=1,weight_filler=conv_filler, bias_filler=const_filler):
    """Accepts same parameters as convolutional layer, but everything is backwards
    https://github.com/carpedm20/DCGAN-tensorflow/blob/master/ops.py#L72 <--- (???)
    """
    deconv = L.Deconvolution(bottom,
                             kernel_size=ks,
                             num_output=nout,
                             stride=stride,
                             pad=pad,
                             group=group,
                             param=learned_param,
                             weight_filler=weight_filler,
                             bias_filler=bias_filler)
    bn = L.BatchNorm(deconv,param=[dict(lr_mult=0)] * 3) # what is this @Vash
    relu = L.ReLU(bn, in_place=True)
    return deconv, relu

In [ ]:
def deconv2d_tanh(bottom, ks, nout, stride=2, pad=1, group=1,weight_filler=conv_filler, bias_filler=const_filler):
    """Accepts same parameters as convolutional layer, but everything is backwards
    https://github.com/carpedm20/DCGAN-tensorflow/blob/master/ops.py#L72 <--- (???)
    """
    deconv = L.Deconvolution(bottom,
                             kernel_size=ks,
                             num_output=nout,
                             stride=stride,
                             pad=pad,
                             group=group,
                             param=learned_param,
                             weight_filler=weight_filler,
                             bias_filler=bias_filler)
    tanh = L.TanH(deconv, in_place=True)
    return deconv, relu

In [ ]:
def discriminator(data):
    """Discriminator in the DC-GAN setup. Originally follows TensorFlow implementation 
    https://github.com/carpedm20/DCGAN-tensorflow/blob/master/model.py#L184
    
    Python syntax based on:
    https://github.com/BVLC/caffe/blob/master/examples/pycaffe/caffenet.py
    
    Args:
        data: an input image, either from the original dataset or the generator
    
    Returns:
        Caffe net prototxt
    
    """
    cov0, relu0 = conv_relu(data, ks=5, nout=df_dim, stride=2)
    conv1, relu1 = conv_relu_with_batchnorm(relu0, ks=5, nout=df_dim * 2, stride=2)
    conv2, relu2 = conv_relu_with_batchnorm(relu1, ks=5, nout=df_dim * 4, stride=2)
    conv3, relu3 = conv_relu_with_batchnorm(relu2, ks=5, nout=df_dim* 8 , stride=2)
    fc4, sigmoid4 = fc_sigmoid(n.relu3, 1)
                       
    return to_proto(sigmoid4)

In [ ]:
def generator(z, y):
    """Generator in DC-GAN setup. Originally follows TensorFlow implementation
    https://github.com/carpedm20/DCGAN-tensorflow/blob/master/model.py#L212
    They seem to be conditioning every step of the way
    
    Args:
        z: input noise, generated by sampler
        y: conditioning vector, a caption embedding
    
    Returns:
        Caffe net prototxt
    """
    z = np.concatenate(z, y) #forms one long vector out of z, y
    # pre-processing to turn into a larger feature vector
    fc1 = fc_linear(z, gf_dim * 8 * 4 * 4)
    relu1 = L.ReLU(L.BatchNorm(reshape(fc1,-1, 4, 4, gf_dim * 8), param=[dict(lr_mult=0)] * 3), in_place=True)
    
    deconv2, relu2 = deconv2d_relu_batchnorm(relu1, ks=8, nout=df_dim *4, stride=2)
    deconv3, relu3 = deconv2d_relu_batchnorm(relu2, ks=16, nout=df_dim *2, stride=2)
    deconv4, relu4 = deconv2d_relu_batchnorm(relu3, ks=32, nout=df_dim, stride=2)
    
    tanh5 = deconv2d_tanh(relu4, ks=64, nout=3, stride=2)
    
    return to_proto(tanh5)
    
    
    
    
    
    
    
    
    
    
    